# Test balances, swaps and superswaps

Make sure honey is running via `python honey.py`

In [ ]:
from sugar import AsyncOPChainSimnet, OPChainSimnet, AsyncOPChain, AsyncBaseChain
from sugar import get_simnet_chain_from_token, get_async_simnet_chain_from_token
from sugar.superswap import MockSuperswapRelayer, AsyncSuperswap, Superswap
from sugar.helpers import require_supersim, atime_it
from fastcore.test import test_eq
from test_specs import superswap_tests_specs

require_supersim()

## Check on balances

In [ ]:
# quick balance spot check (and test get_token_balance in the process)

async with AsyncOPChainSimnet() as simnet:
    test_eq(await simnet.get_token_balance(AsyncOPChainSimnet.velo), 1000000000000000000000)
    test_eq(await simnet.get_token_balance(AsyncOPChainSimnet.eth), 10000000000000000000000)

with OPChainSimnet() as simnet:
    test_eq(simnet.get_token_balance(OPChainSimnet.velo), 1000000000000000000000)
    test_eq(simnet.get_token_balance(OPChainSimnet.eth), 10000000000000000000000)

## Test swaps

In [ ]:
# velo -> usdc
async with AsyncOPChain() as op:
    async with atime_it("Get quote"):
        quote = await op.get_quote(from_token=op.velo, to_token=op.usdc, amount=op.velo.parse_units(10))

    async with AsyncOPChainSimnet() as op_simnet:
        print(await op_simnet.swap_from_quote(quote, slippage=0.1))

    with OPChainSimnet() as op_simnet_sync:
        print(op_simnet_sync.swap_from_quote(quote, slippage=0.1))

# velo -> eth
async with AsyncOPChain() as op:
    async with atime_it("Get quote"):
        quote = await op.get_quote(op.velo, op.eth, op.velo.parse_units(10))

    async with AsyncOPChainSimnet() as op_simnet:
        print(await op_simnet.swap_from_quote(quote, slippage=0.1))

# eth -> velo
async with AsyncOPChain() as op:
    async with atime_it("Get quote"):
        quote = await op.get_quote(op.eth, op.velo, op.eth.parse_units(0.001))

    async with AsyncOPChainSimnet() as op_simnet:
        print(await op_simnet.swap_from_quote(quote, slippage=0.1))


Get quote took 3.5471 seconds
AttributeDict({'type': 2, 'status': 1, 'cumulativeGasUsed': 424155, 'logs': [AttributeDict({'address': '0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db', 'topics': [HexBytes('0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925'), HexBytes('0x000000000000000000000000f39fd6e51aad88f6f4ce6ab8827279cfffb92266'), HexBytes('0x00000000000000000000000001d40099fcd87c018969b0e8d4ab1633fb34763c')], 'data': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'), 'blockHash': HexBytes('0x4e4349bc0e26a4303a262a2dd4021d74bd3c53cf10703694daf9f1ac3e1ba665'), 'blockNumber': 139266624, 'blockTimestamp': '0x688deea9', 'transactionHash': HexBytes('0x50642cff1c1116dad79ffdfa6f8bf03e1bcd9a78bf288d231509c8eb904f0e99'), 'transactionIndex': 0, 'logIndex': 0, 'removed': False}), AttributeDict({'address': '0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db', 'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'), HexBytes('

## Superswaps

In [ ]:
# try unsupported chains

with OPChainSimnet() as op_sim:
    from_token, to_token = AsyncOPChain.velo, AsyncBaseChain.aero 
    error = None
    try:
        tx = Superswap(chain_for_writes=op_sim, relayer=MockSuperswapRelayer()).swap(from_token, to_token, amount=from_token.parse_units(20))
    except ValueError as e:
        error = e
    test_eq(str(error), "Superswap only supports ['OP', 'Lisk', 'Uni']. Got OP -> Base")
     

In [ ]:
# sync version

for from_token, to_token, amount, requires_relay in superswap_tests_specs:
    with get_simnet_chain_from_token(from_token) as from_sim:
        relayer=MockSuperswapRelayer()
        tx = Superswap(chain_for_writes=from_sim, relayer=relayer).swap(from_token, to_token, amount=amount, slippage=0.1)
        assert(tx.startswith("0x"))

        if not requires_relay: 
            test_eq(relayer.get_call_count(), 0)
            continue

        test_eq(relayer.get_call_count(), 1)
        last_call = relayer.get_last_call()
        test_eq(type(last_call["salt"]), str)
        test_eq(type(last_call["origin_domain"]), int)

Mock relayer received call: {'calls': [{'to': '0x0000000000000000000000001217bfe6c773eec6cc4a38b5dc45b92292b6e189', 'value': '0', 'data': '0x095ea7b300000000000000000000000001d40099fcd87c018969b0e8d4ab1633fb34763cffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff'}, {'to': '0x00000000000000000000000001d40099fcd87c018969b0e8d4ab1633fb34763c', 'value': '0', 'data': '0x24856bc3000000000000000000000000000000000000000000000000000000000000004000000000000000000000000000000000000000000000000000000000000000800000000000000000000000000000000000000000000000000000000000000002a1a100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000040000000000000000000000000000000000000000000000000000000000000032000000000000000000000000000000000000000000000000000000000000002c00000000000000000000000000000000000000000000000000000000000000040000000000000000000000000000000000

In [ ]:
# async version

for from_token, to_token, amount, requires_relay in superswap_tests_specs:
    async with get_async_simnet_chain_from_token(from_token) as from_sim:
        relayer=MockSuperswapRelayer()
        tx = await AsyncSuperswap(chain_for_writes=from_sim, relayer=relayer).swap(from_token, to_token, amount=amount, slippage=0.1)
        assert(tx.startswith("0x"))

        if not requires_relay: 
            test_eq(relayer.get_call_count(), 0)
            continue

        test_eq(relayer.get_call_count(), 1)
        last_call = relayer.get_last_call()
        test_eq(type(last_call["salt"]), str)
        test_eq(type(last_call["origin_domain"]), int)

Mock relayer received call: {'calls': [{'to': '0x0000000000000000000000001217bfe6c773eec6cc4a38b5dc45b92292b6e189', 'value': '0', 'data': '0x095ea7b300000000000000000000000001d40099fcd87c018969b0e8d4ab1633fb34763cffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff'}, {'to': '0x00000000000000000000000001d40099fcd87c018969b0e8d4ab1633fb34763c', 'value': '0', 'data': '0x24856bc3000000000000000000000000000000000000000000000000000000000000004000000000000000000000000000000000000000000000000000000000000000800000000000000000000000000000000000000000000000000000000000000002a1a100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000040000000000000000000000000000000000000000000000000000000000000032000000000000000000000000000000000000000000000000000000000000002c00000000000000000000000000000000000000000000000000000000000000040000000000000000000000000000000000